# ปัญหาการจัดกลุ่มลูกค้า


จากข้อมูลขายของออนไลน์ที่นักศึกษาเคยใช้ จะเห็นว่ามีรายการสินค้าที่ลูกค้าสั่งซื้อเป็นจำนวนมาก  จึงเกิดความสนใจที่จะจัดลูกค้าเป็นกลุ่ม ๆ (**Customer segmentation**) เพื่อที่ว่า เราอาจค้นพบลูกค้ากลุ่มที่เข้ามาซื้อบ่อย แต่ยอดเงินที่จ่ายไม่สูง หรือลูกค้ากลุ่มที่นาน ๆ ครั้งเข้ามาซื้อ แต่คิดเป็นยอดซื้อที่สูง เป็นต้น เพื่อที่ว่าเราจะสามารถจัดโปรโมชันหรือการส่งเสริมการขายได้ถูกกลุ่ม ซึ่งเป็นไปได้ว่าอาจมีนักช๊อปที่ยังไม่ช๊อปแต่รอโปรโมชันอยู่ก็เป็นได้

**จุดประสงค์การเรียนรู้**

1. สามารถคำนวณค่า Recency (R), Frequency (F) และ Monetary (M) จากข้อมูลได้
2. สามารถใช้วิธี K-Means ในการแบ่งกลุ่มข้อมูลลูกค้าจากค่า RFM
3. สามารถตีความผลการจัดกลุ่มที่ได้จาก K-Means

สำหรับหัวข้อนี้ เราจะใช้ข้อมูลการซื้อของออนไลน์ ([Online Retail](https://archive.ics.uci.edu/ml/datasets/online+retail)) ซึ่งเป็นตัวเดียวกันกับที่เคยใช้ในคาบก่อน ๆ แต่มีจำนวนของข้อมูลที่มากกว่า




โดยรายละเอียดของข้อมูลแต่ละคอลัมน์ มีดังนี้

| คอลัมน์ | คำอธิบาย |
| :-- | :-- |
| `InvoiceNo` | เลขธุรกรรม (ตัวเลข 6 หลัก) โดยหากมีการขึ้นต้นด้วย `C` จะหมายถึงการยกเลิกสินค้า |
| `StockCode` | รหัสสินค้า (ตัวเลข 5 หลัก) |
| `Description` | ชื่อสินค้า |
| `Quantity` | จำนวนชิ้นที่ซื้อ  |
| `InvoiceDate` | วันและเวลาของธุรกรรม  |
| `UnitPrice` | ราคาต่อหน่วย |
| `CustomerID` | รหัสลูกค้า (ตัวเลข 5 หลัก) |
| `Country` | ประเทศที่อยู่ของลูกค้า |


## อ่านข้อมูล

ก่อนอื่น อ่านข้อมูลและทำการแปลงชนิดข้อมูลให้เหมาะสม



In [1]:
!wget -c -q https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx -O online_retail.xlsx

^C


In [2]:
%reset -f
import pandas as pd
df = pd.read_excel('online_retail.xlsx', sheet_name='Online Retail')  # การโหลดค่อนข้างช้า เพราะไฟล์มีขนาดใหญ่มาก

In [3]:
df['InvoiceNo'] = df['InvoiceNo'].astype('string')
df['StockCode'] = df['StockCode'].astype('string')
df['Description'] = df['Description'].astype('string')
df['CustomerID'] = df['CustomerID'].astype('Int64').astype('string')  # Int64 can have NaN integer and also used to remove floating point
df['Country'] = df['Country'].astype('string')

In [4]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
481672,577358,82578,KITCHEN METAL SIGN,4,2011-11-18 15:59:00,1.25,<NA>,United Kingdom
351759,567668,23328,SET 6 SCHOOL MILK BOTTLES IN CRATE,2,2011-09-21 15:29:00,7.46,<NA>,United Kingdom
341370,566757,20956,PORCELAIN T-LIGHT HOLDERS ASSORTED,1,2011-09-14 15:54:00,2.46,<NA>,United Kingdom
487167,577768,20727,LUNCH BAG BLACK SKULL.,2,2011-11-21 15:18:00,4.13,<NA>,United Kingdom
519862,580188,23329,DECORATIVE WICKER HEART LARGE,9,2011-12-02 12:02:00,1.65,16440,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  string        
 1   StockCode    541909 non-null  string        
 2   Description  540455 non-null  string        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  string        
 7   Country      541909 non-null  string        
dtypes: datetime64[ns](1), float64(1), int64(1), string(5)
memory usage: 33.1 MB


ชุดข้อมูลนี้มี Customer กี่คน?

In [6]:
df.groupby('CustomerID').ngroups

4372

ชุดข้อมูลนี้มีกี่ Invoice?


In [7]:
df.groupby('InvoiceNo').ngroups

25900

## เตรียมข้อมูล

ข้อมูล Online Retail เป็นข้อมูลที่ยังไม่ได้ทำความสะอาดดีนัก ซึ่งในทางปฏิบัติจำเป็นจะต้องทำ Exploratory Data Analysis ก่อน เพื่อทำความเข้าใจข้อมูล อย่างไรก็ตาม จะขอข้ามขั้นตอนนั้น เพราะถือว่าได้เคยทำไปแล้วในคาบก่อน ๆ ดังนั้น ในส่วนของการเตรียมข้อมูล จะกระทำตามรายการต่อไปนี้

1. เอาข้อมูลเฉพาะคำสั่งซื้อที่มาจากอังกฤษ (United Kingdom) เพราะพบว่าคำสั่งซื้อส่วนมากมาจากที่นี่ นอกจากนี้ พฤติกรรมของลูกค้าแต่ละประเทศอาจต่างกัน ดังนั้นการเลือกสร้างโมเดลเฉพาะของอังกฤษ ก็สมเหตุสมผล
2. ลบแถวที่ `CustomerID` เป็น `NaN` เพราะเราต้องการจัดกลุ่มลูกค้า หากไม่มีก็ทำอะไรต่อไม่ได้ เพราะไม่ทราบว่าใครซื้อสินค้า
3. ข้อมูลมีแค่ `UnitPrice` และ `Quantity` ดังนั้นจึงจะสร้างคอลัมน์ใหม่ที่แสดงจำนวนเงินที่ลูกค้าจ่าย ตั้งชื่อคอลัมน์ว่า `TotalPrice`

In [8]:
df = df[df['Country'] == 'United Kingdom'].copy() # 1) พิจารณาอังกฤษเท่านั้น
df.dropna(axis=0, subset=['CustomerID'], inplace=True)  # 2) ลบแถวที่ไม่มี CustomerID
df['TotalPrice'] = df['Quantity']*df['UnitPrice']    # 3) จำนวนเงินที่ลูกค้าจ่ายในรายการนั้น

## การคำนวณค่า RFM จากข้อมูล

ก่อนอื่น RFM คืออะไร?

RFM เป็นค่า 3 ค่าที่อธิบายคุณลักษณะของลูกค้า ซึ่งมีค่าดังนี้

1. **R = Recency** หมายถึง ลูกค้าเข้ามาซื้อของล่าสุดเมื่อไร ยิ่งค่าน้อย ก็แปลว่าลูกค้าเพิ่งเข้ามาซื้อไม่นาน
2. **F = Frequency** หมายถึง ความถี่ของการซื้อสินค้า
3. **M - Monetary value** หมายถึง จำนวนเงินที่ลูกค้าจ่าย

ค่าทั้ง 3 ค่านี้ ถูกเลือกมาใช้ในการประเมินลักษณะของลูกค้า อย่างไรก็ตาม ในทางปฏิบัติ การคำนวณค่าเหล่านี้ไม่มีสูตรตายตัว มีวิธีอยู่หลายวิธี ซึ่งวิธีการคำนวณในหัวข้อต่อไป เป็นเพียงวิธีการแบบหนึ่งเท่านั้น

ก่อนอื่น แบ่งตารางตามรหัสลูกค้า

In [9]:
customer_group = df.groupby(['CustomerID'])

### การคำนวณ Recency

พิจารณาจากคอลัมน์ `InvoiceDate` ซึ่งเป็นตัวแปร DateTime อยู่แล้ว โดยจะคำนวณผลต่างระหว่างวันที่ล่าสุดในชุดข้อมูล กับ `InvoiceDate` ล่าสุดของลูกค้าแต่ละคน

โดยผลลัพธ์จะเก็บเป็นจำนวนวัน และหากจำนวนเกิน 365 วัน จะให้ถือเป็นค่า R = 365

In [10]:
today = df['InvoiceDate'].max()  # วันที่ล่าสุดในชุดข้อมูล

def days_since_last_invoice(inv_date):     # inv_date เป็นคอลัมน์ของเฉพาะ customer แต่ละคน
    return (today - inv_date.max()).days   # จำนวนเดือน

R = customer_group['InvoiceDate'].agg(days_since_last_invoice)
R.name = 'Recency'
R

CustomerID
12346    325
12747      1
12748      0
12749      3
12820      2
        ... 
18280    277
18281    180
18282      7
18283      3
18287     42
Name: Recency, Length: 3950, dtype: int64

In [11]:
R = R.clip(lower=0, upper=365)
R

CustomerID
12346    325
12747      1
12748      0
12749      3
12820      2
        ... 
18280    277
18281    180
18282      7
18283      3
18287     42
Name: Recency, Length: 3950, dtype: int64

### การคำนวณ Frequency

ในที่นี้ จะประเมินความถี่ของการที่ลูกค้ามีต่อร้านขายของออนไลน์จากจำนวน Invoice ที่เกิดขึ้นของลูกค้าคนหนึ่ง โดยให้คะแนน 0 ถึง 10

- ถ้าจำนวน Invoice น้อยกว่า 10 ก็จะให้คะแนนเท่ากับจำนวน Invoice
- ถ้าจำนวน Invoice มากกว่า 10 ก็จะให้คะแนน 10 คะแนน

อนึ่ง ในทางปฏิบัติควรพิจารณาเฉพาะ Invoice ที่ไม่เก่าเกินไป เช่น Invoice ที่อายุไม่เกิน 10 เดือน แต่สำหรับในที่นี้ จะพิจารณาจำนวน Invoice ทั้งหมดของลูกค้าแต่ละรายโดยไม่สนว่าเก่าหรือใหม่

In [12]:
F = customer_group['InvoiceNo'].nunique()
F = F.clip(lower=0, upper=10)
F.name = 'Frequency'
F

CustomerID
12346     2
12747    10
12748    10
12749     8
12820     4
         ..
18280     1
18281     1
18282     3
18283    10
18287     3
Name: Frequency, Length: 3950, dtype: int64

### การคำนวณ Monetary value

จำนวนเงินรวมที่ลูกค้าจ่ายเพื่อซื้อสินค้า โดยปกติจะต้องกำหนดกรอบเวลาของการประเมินมูลค่าให้ชัดเจน เช่นไม่เกิน 1 ปี เป็นต้น แต่เพื่อความง่าย จะคำนวณ Monetary value จากยอดรวมทั้งหมดของลูกค้าแต่ละคนในชุดข้อมูลที่มี (เนื่องจากข้อมูลเรามีย้อนหลังไม่นาน)

In [13]:
M = customer_group['TotalPrice'].sum()
M.name = 'Monetary'
M

CustomerID
12346        0.00
12747     4196.01
12748    29072.10
12749     3868.20
12820      942.34
           ...   
18280      180.60
18281       80.82
18282      176.60
18283     2094.88
18287     1837.28
Name: Monetary, Length: 3950, dtype: float64

In [14]:
M.describe()

count      3950.000000
mean       1713.385669
std        6548.608224
min       -4287.630000
25%         282.255000
50%         627.060000
75%        1521.782500
max      256438.490000
Name: Monetary, dtype: float64

อย่างไรก็ตาม จากคำสั่ง `describe()` จะพบว่าค่ายอดรวมของลูกค้าบางรายติดลบ เราสันนิษฐานว่า ยอดติดลบนี้มาจากการคืนสินค้า ทำให้มีการลงยอดซื้อเป็นลบ โดยปกติแล้ว หากมียอดซื้อติดลบ ก็ควรจะมียอดซื้อเป็นบวกปรากฎด้วย อย่างไรก็ตาม เนื่องจากชุดข้อมูลที่เรามีอาจไม่ได้ย้อนหลังไปครอบคลุมครบถ้วนทุกการสั่งซื้อ ดังนั้นเราจะทำตัดค่า Monetary ที่เป็นค่าลบออกไป

In [15]:
M = M[M > 0]

นอกจากนี้ ค่ายอดใช้จ่ายที่มากเกินไป ก็สะท้อนว่าลูกค้าคนนั้นอาจเป็น outlier ในทางสถิติ เราอาจพิจารณาว่า หากยอดใช้จ่ายมีค่ามากกว่าค่าเฉลี่ยบวกลบ 2.5 เท่าของส่วนเบี่ยงเบนมาตรฐาน ขั้นไป จะถือว่ายอดใช้จ่ายนั้น เป็น outlier

เราสามารถเลือกเอาแต่ค่าในช่วง mean±2.5×SD ดังนี้

In [16]:
z_scores = (M - M.mean())/M.std()
M = M[z_scores.abs() < 2.5]

### คุณลักษณะ RFM

จากค่า R, F และ M ที่คำนวณได้ก่อนหน้า เราจะนำมารวมกันเป็น DataFrame ดังนี้

In [17]:
RFM = pd.concat((R,F,M), axis=1) # ค่าของ Series R, F และ M ที่มี CustomerID เดียวกัน จะถูกนำมาต่อกันเป็น 1 แถวของ DataFrame RFM
RFM

,Recency,Frequency,Monetary
CustomerID,,,
12346,325,2,NaN
12747,1,10,4196.01
12748,0,10,NaN
12749,3,8,3868.20
12820,2,4,942.34
...,...,...,...
18280,277,1,180.60
18281,180,1,80.82
18282,7,3,176.60


ปรากฎว่า RFM มี NaN อยู่ในคอลัมน์ Monetary ซึ่งเป็นผลจากการที่เราตัดเอา M <= 0 และ M ที่เป็น outlier ออกไป ดังนั้น เราจะลบแถวที่มี NaN ทิ้งทั้งหมด

In [18]:
RFM.isnull().sum()

Recency       0
Frequency     0
Monetary     81
dtype: int64

In [19]:
RFM.dropna(inplace=True)

## Standardization

ในขั้นตอนต่อไป เป็นขั้นตอนการ Standardize ค่า RFM ซึ่งถือเป็นคุณลักษณะ 3 คุณลักษณะ (feature) ที่เราจะใช้เทรนโมเดล

In [20]:
from sklearn import set_config
set_config(transform_output='pandas')  # ให้ Scikit-learn ใช้ Pandas ในการเก็บข้อมูลแทน Numpy ที่เป็นค่า default

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

RFM_normalized = scaler.fit_transform(RFM)
RFM_normalized

,Recency,Frequency,Monetary
CustomerID,,,
12747,-0.900523,2.053571,1.505387
12749,-0.880237,1.393528,1.334654
12820,-0.890380,0.073442,-0.189223
12821,1.249845,-0.916622,-0.631731
12822,-0.200639,-0.256579,-0.201390
...,...,...,...
18280,1.899013,-0.916622,-0.585960
18281,0.915118,-0.916622,-0.637929
18282,-0.839664,-0.256579,-0.588044


## K-Means Clustering

จากข้อมูลที่เตรียมได้จากกระบวนการที่ผ่านมา จะนำไปจัดกลุ่ม (clustering) ด้วยวิธีการ K-Means clustering ซึ่งหลักการทำงานศึกษาได้จากสไลด์ต่อไปนี้ ([ลิงค์](https://docs.google.com/presentation/d/1LC-R_n-aIEvvhtg7WUpbi-rNul1wV-Tc7qdGypNtnZM/edit?usp=sharing))

K-Means clustering เป็นโมเดลที่แตกต่างจากโมเดล Linear Regression, Logistic Regression และ Decision Tree Classifier ในแง่ที่ว่า เราสอนโมเดล K-Means โดยไม่ได้บอกเป้าหมาย (target) ที่ถูกต้องให้กับโมเดล ความแตกต่างในแง่นี้ จึงมีการจัดประเภทของโมเดล เป็น 2 ประเภท ดังนี้

1. **การเรียนรู้แบบมีผู้สอน (Supervised learning)** โมเดลในกลุ่มนี้ ได้แก่ Linear Regression, Logistic Regression และ Decision Tree Classifier ซึ่งข้อมูลที่จะนำมาสอนโมเดลประเภทนี้ต้องประกอบด้วย X (คุณลักษณะ) ที่คู่กับ Y (เอาต์พุตที่ถูกต้อง)
2. **การเรียนรู้แบบไม่มีผู้สอน (Unsupervised learning)**  โมเดลในกลุ่มนี้ ได้แก่ K-Means clustering ซึ่งข้อมูลที่นำมาสอนโมเดลประเภทนี้ จะมีเพียง X (คุณลักษณะ) เท่านั้น แล้วตัวโมเดลจะพยายามจัดกลุ่มของ X ให้

สำหรับโมเดล K-Means ใน Scikit-learn เรียกใช้ได้จาก

```python
from sklearn.cluster import KMeans
```

ซึ่งสามารถสร้างโมเดลเปล่า ๆ ก่อนการเทรน ดังนี้

```python
model = KMeans(n_clusters=จำนวนกลุ่มที่ต้องการแบ่ง,      
                init='k-means++',  # ระบุเทคนิคชื่อ k-means++ ที่ช่วยกำหนดตำแหน่ง centroid ตั้งต้น
                n_init='auto',
                max_iter=1000,
                random_state=123)
```

โดยการเทรน สามารถทำได้โดยใช้

```python
model.fit(ข้อมูล)   # ไม่ใช่ model.fit(ข้อมูล, เป้าหมาย) เหมือนโมเดลอื่น
```
ซึ่งเมื่อเทรนแล้ว จะได้จำนวน Cluster ตามที่กำหนดไว้แต่แรก โดยแต่ละ Cluster จะมีจุดศูนย์กลางของ Cluster คือ centroid ที่ดูได้จาก `model.cluster_centers_`

อย่างไรก็ตาม เวลานำโมเดลไปใช้จริง เราจะใช้คำสั่ง `model.predict(ข้อมูล)` ซึ่งจะให้ผลลัพธ์ว่า ข้อมูลแต่ละจุด อยู่ใกล้ centroid ใดมากที่สุด หรือก็คือ ให้ผลทำนายว่าข้อมูลแต่ละจุดเป็นสมาชิกของ Cluster ใด

ตัวอย่างการเขียนโค้ด เป็นดังนี้

In [21]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=6, init='k-means++', n_init='auto', random_state=123)
model.fit(RFM_normalized)

KMeans(n_clusters=6, n_init='auto', random_state=123)

In [22]:
model.cluster_centers_  # แต่ละแถวคือ centroid ของ 1 cluster

array([[-0.66854588,  1.81049389,  0.96500007],
       [-0.48978925, -0.65586107, -0.41935455],
       [-0.73611446,  1.93878083,  4.3255557 ],
       [ 0.78052503, -0.55735111, -0.41707422],
       [-0.54964521,  0.44406955,  0.03406537],
       [ 2.0731158 , -0.77518417, -0.51054402]])

จะนั้น เมื่อลองทำนาย จะเห็นว่าโมเดลใช้ค่า RFM ที่มี มาจัดกลุ่มว่า `CustomerID` หนึ่ง ๆ นั้นจัดเป็นสมาชิก Cluster ใด

In [23]:
pred = model.predict(RFM_normalized)
RFM['Cluster'] = pred     # เราเอาผลการทำนายมาใส่ในตาราง RFM เดิมก่อน normalized ก็เพราะ เราจะได้เห็นค่าที่จริง ๆ ที่ไม่ใช่ค่า normalized
RFM.head()

,Recency,Frequency,Monetary,Cluster
CustomerID,,,,
12747,1,10,4196.01,0
12749,3,8,3868.20,0
12820,2,4,942.34,4
12821,213,1,92.72,3
12822,70,3,918.98,1


จากผลลัพธ์ที่ได้ข้างต้น ลองดูว่า

- แต่ละ Cluster มีค่า RMF แต่ละค่า เฉลี่ยเท่ากับเท่าไร
- แต่ละ Cluster มีจำนวนลูกค้ากี่ราย

In [24]:
cluster_summary = (
    RFM.groupby('Cluster')
       .agg({'Recency':'mean', 'Frequency':'mean', 'Monetary':'mean','Cluster':'count'})
       .rename(columns={'Cluster':'Number of customers'})
       .reset_index()
)
cluster_summary

,Cluster,Recency,Frequency,Monetary,Number of customers
0,0,23.870130,9.263451,3158.461076,539
1,1,41.493344,1.790133,500.485146,1277
2,2,17.208696,9.652174,9610.764435,115
3,3,166.622705,2.091820,504.716830,599
4,4,35.512077,5.124396,1372.208492,828
5,5,294.164384,1.428571,325.400372,511


ซึ่งสามารถนำมาวาดกราฟเปรียบเทียบค่า RFM เฉลี่ยแต่ละ Cluster ดังนี้

In [25]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

col_names = ['Recency','Frequency','Monetary']
fig = make_subplots(rows=1, cols=3, subplot_titles=col_names)
for i,col in enumerate(col_names):
  fig.add_trace(
      go.Bar(x=cluster_summary['Cluster'], y=cluster_summary[col], name=col),
      row=1, col=i+1
  )
fig.update_xaxes(tickmode='array', tickvals = [0,1,2,3,4,5])
fig.update_layout(showlegend=False)
fig.show()

จากผลลัพธ์ที่ได้ เราอาจสนใจทำการตลาดส่งเสริมการขาย ด้วยโปรโมชันที่แตกต่างกันในแต่ละกลุ่ม

- **Cluster 0 และ 2** อาจยุบรวมเป็นกลุ่มเดียวกัน (Monetary > 2k) เป็นรายที่จ่ายเยอะ  และมาซื้อบ่อย  อาจเป็นผู้ซื้อสินค้าเพื่อนำไปขายต่อ
- **Cluster 4** มาล่าสุด 35 วันโดยเฉลี่ย (Recency=35) ซึ่งถือว่าค่อนข้างไม่บ่อยเท่า Cluster 0 และ 2 และกำลังซื้อถือว่าอยู่ในระดับกลาง ๆ อาจกระตุ้นกลุ่มนี้มาซื้อบ่อยขึ้นและใช้จ่ายมากขึ้น
- **Cluster 1, 3 และ 5** โดยเฉลี่ยซื้อไม่บ่อย และยอดซื้อไม่มาก (ราว ๆ 300-500 ปอนด์)
  - Cluster 1: เพิ่งเข้ามาไม่นาน (เฉลี่ย 41 วัน)  
  - Cluster 3: ไม่ได้เข้ามาเกือบ 6 เดือน (เฉลี่ย 166 วัน)
  - Cluster 5: ไม่ได้เข้ามาเกือบ 10 เดือน (เฉลี่ย 294 วัน)  




**เพิ่มเติม**

จากตาราง `cluster_summary` เราอาจแปลงค่า R, F, และ M ซึ่งเป็นค่าตัวเลขทศนิยม ให้เป็นค่าระดับได้ โดยใช้คำสั่ง `pd.cut()` ([คู่มือ](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html))

ยกตัวอย่างในโค้ดต่อไปนี้ เป็นการแปลงค่า Recency ตามตาราง

| ช่วงของค่า Recency | แปลงเป็นค่า |
|:--|:--|
| 0 < R ≤ 60 | 2 months |
| 60 < R ≤ 180 | 6 months |
| 180 < R ≤ ∞ | >6 months |

In [26]:
import numpy as np
cluster_summary['R'] = pd.cut(cluster_summary['Recency'],
                              bins=[0, 60, 180, np.inf],  # np.inf คือค่า Infinity
                              labels=['2 months', '6 months', '>6 months'])
cluster_summary

,Cluster,Recency,Frequency,Monetary,Number of customers,R
0,0,23.870130,9.263451,3158.461076,539,2 month
1,1,41.493344,1.790133,500.485146,1277,2 month
2,2,17.208696,9.652174,9610.764435,115,2 month
3,3,166.622705,2.091820,504.716830,599,6 months
4,4,35.512077,5.124396,1372.208492,828,2 month
5,5,294.164384,1.428571,325.400372,511,>6 months


ในลักษณะเดียวกัน เราสามารถทำกับค่า Frequency และ Monetary ได้

In [27]:
cluster_summary['F'] = pd.cut(cluster_summary['Frequency'],
                              bins=[0, 3, 8, np.inf],
                              labels=['3 times', '8 times', '>8 times'])
cluster_summary['M'] = pd.cut(cluster_summary['Monetary'],
                              bins=[0, 1000, 2000, 4000, np.inf],
                              labels=['low', 'medium', 'high', 'very high'])
cluster_summary.sort_values(by=['Recency'], ascending=[True])  # เรียงลำดับให้ดูง่าย

,Cluster,Recency,Frequency,Monetary,Number of customers,R,F,M
2,2,17.208696,9.652174,9610.764435,115,2 month,>8 times,very high
0,0,23.870130,9.263451,3158.461076,539,2 month,>8 times,high
4,4,35.512077,5.124396,1372.208492,828,2 month,8 times,medium
1,1,41.493344,1.790133,500.485146,1277,2 month,3 times,low
3,3,166.622705,2.091820,504.716830,599,6 months,3 times,low
5,5,294.164384,1.428571,325.400372,511,>6 months,3 times,low


>**Note:** บทความวิจัยที่ใช้ RFM ในปัญหา Customer segmentation ได้แก่ "Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining" ดูที่ [Link](https://link.springer.com/article/10.1057/dbm.2012.17)